In [19]:
import os
import openai
from dotenv import load_dotenv
import ast
import json

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
with open("training.jsonl", "rb") as file:
    result = openai.File.create(file=file, purpose="fine-tune")
print(result)

{
  "object": "file",
  "id": "file-AARcZzH64IegrDxmCzMFjEjC",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 7246,
  "created_at": 1697131334,
  "status": "uploaded",
  "status_details": null
}


In [4]:
tune_result = openai.FineTuningJob.create(
    training_file=result.get("id"),
    model="gpt-3.5-turbo",
    suffix="etymology",
)
print(tune_result)

{
  "object": "fine_tuning.job",
  "id": "ftjob-g5Rl6cUeC0NzKPH5DgzIufd9",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697131527,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-9FaMn9oaWBnDYWd8gZY5OfMa",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-AARcZzH64IegrDxmCzMFjEjC",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}


In [7]:
status = openai.FineTuningJob.retrieve(tune_result.get("id"))
print(status)

{
  "object": "fine_tuning.job",
  "id": "ftjob-g5Rl6cUeC0NzKPH5DgzIufd9",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697131527,
  "finished_at": 1697131894,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal:etymology:88tp2w4g",
  "organization_id": "org-9FaMn9oaWBnDYWd8gZY5OfMa",
  "result_files": [
    "file-Woox58UhfxO3IX4n0GPbT3iu"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-AARcZzH64IegrDxmCzMFjEjC",
  "hyperparameters": {
    "n_epochs": 10
  },
  "trained_tokens": 17380,
  "error": null
}


In [14]:
completion, *_ = openai.ChatCompletion.create(
    model=status.get("fine_tuned_model"),
    messages=[
        {"role": "system", "content": "Respond with the etymology of the following word"},
        {"role": "user", "content": "story"},
    ]
).choices
print(completion.message.content)

{'Word': 'story', 'Word_Origin': 'Middle English from Old French', 'Root_Word': 'estoire', 'Derivation': "From Old French 'estoire' meaning 'chronicle, history'", 'Morphological_Variants': ['stories', 'storyteller'], 'Cultural_Context': 'Literature, entertainment, journalism', 'Syntactic_Role': 'Noun', 'Related_Words': ['narrative', 'tale', 'plot'], 'Definition': 'A narrative or tale', 'Synonyms': ['account', 'narrative'], 'Antonyms': ['fact'], 'Phonetic': '/ˈstɔːri/'}


In [18]:
json.dumps(ast.literal_eval(completion.message.content))

'{"Word": "story", "Word_Origin": "Middle English from Old French", "Root_Word": "estoire", "Derivation": "From Old French \'estoire\' meaning \'chronicle, history\'", "Morphological_Variants": ["stories", "storyteller"], "Cultural_Context": "Literature, entertainment, journalism", "Syntactic_Role": "Noun", "Related_Words": ["narrative", "tale", "plot"], "Definition": "A narrative or tale", "Synonyms": ["account", "narrative"], "Antonyms": ["fact"], "Phonetic": "/\\u02c8st\\u0254\\u02d0ri/"}'